In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


image_size = 224
input_shape = (image_size, image_size, 3)
batch_size = 32

dataset_root = "/kaggle/input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = os.path.join(dataset_root, "train")
test_dir = os.path.join(dataset_root, "valid")


train_aug = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    shear_range=0.3,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [2]:
train_data = train_aug.flow_from_directory(
    train_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode="categorical"
)

test_aug = ImageDataGenerator(rescale=1/255.0)
test_data = test_aug.flow_from_directory(
    test_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode="categorical"
)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [3]:

base_model = keras.applications.MobileNetV2(
    weights="imagenet", include_top=False, input_shape=input_shape
)


for layer in base_model.layers[-30:]:
    layer.trainable = True

inputs = keras.Input(shape=input_shape)
x = base_model(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
out = layers.Dense(len(train_data.class_indices), activation="softmax")(x)

model = Model(inputs, out)


model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# Train Model
model.fit(train_data, validation_data=test_data, epochs=5)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2197/2197 ━━━━━━━━━━━━━━━━━━━━ 1246s 545ms/step - accuracy: 0.6253 - loss: 1.4923 - val_accuracy: 0.8511 - val_loss: 0.5552
Epoch 2/5
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 850s 386ms/step - accuracy: 0.9564 - loss: 0.1640 - val_accuracy: 0.9491 - val_loss: 0.1679
Epoch 3/5
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 846s 384ms/step - accuracy: 0.9723 - loss: 0.0956 - val_accuracy: 0.9795 - val_loss: 0.0622
Epoch 4/5
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 851s 386ms/step - accuracy: 0.9802 - loss: 0.0702 - val_accuracy: 0.9854 - val_loss: 0.0445
Epoch 5/5
2197/2197 ━━━━━━━━━━━━━━━━━━━━ 854s 387ms/step - accuracy: 0.9836 - loss: 0.0573 - val_accuracy: 0.9790 - val_loss: 0.0631


In [5]:
model.evaluate(test_data)

550/550 ━━━━━━━━━━━━━━━━━━━━ 31s 56ms/step - accuracy: 0.9791 - loss: 0.0658


[0.06306931376457214, 0.9790006875991821]

In [6]:
model.save("plant_disease_detection.h5")

In [7]:
import tensorflow as tf
from tensorflow.keras.models import load_model


model = load_model("/kaggle/working/plant_disease_detection.h5")


converter = tf.lite.TFLiteConverter.from_keras_model(model)


converter.optimizations = [tf.lite.Optimize.DEFAULT]


tflite_model = converter.convert()


with open('plant_disease_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite model saved as 'plant_disease_model.tflite'")

Saved artifact at '/tmp/tmpms_npnxd'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 38), dtype=tf.float32, name=None)
Captures:
  140265329332592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140265329344032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140265329379104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140265329339456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140265329341744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140265329382624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140265329382976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140265329384384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140265329380688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140265329382272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140265329386